# Factors

In this section, we compare the performance of the Industry momentum derived in previous sections with other factors (e.g. Stock Momentum, Value, Size, etc.). In particular, we discuss the Style regression of Sharpe (1992). 

In [ ]:
# hide
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import pandas as pd
from IPython.display import Image, display
from matplotlib import pyplot as plt
from skfin.backtesting import Backtester
from skfin.datasets import load_kf_returns
from skfin.mv_estimators import MeanVariance
from skfin.plot import bar, heatmap, line

## Style analysis

As introduced by Sharpe (1992), Style Analysis is the process of determining what type of investment behaviour an investor or a money manager employs when making investment decisions

Regression to determine the factor exposures $\langle \beta_1,... \beta_K\rangle$ where:

$$ r = \alpha + \beta_1 r_1^{\Phi} + ... + \beta_K r_K^{\Phi} + \epsilon$$

Additional constraints might be added to “regularize" the regression such as non-negative exposures :

$\beta_k \geq 0$ and/or sum equals 1: $\sum_{k=1}^{K} \beta_k = 1$.

Frazzini, Kabiller and Pedersen (2013) state that: 

> Berkshire Hathaway has realized a Sharpe ratio of 0.76, higher than any other stock or mutual fund with a history of more than 30 years, and Berkshire has a significant alpha to traditional risk factors."

How did Warren Buffet do it?

- Use a “style analysis" approach applied to equity factors to
address this question.

The main regression is:
$$r_t = \alpha +\beta_1 MKT_t +\beta_2 SMB_t +\beta_3 HML_t +\beta_4 UMD_t + \beta_5 BAB_t + \beta QMJ_t +\epsilon_t$$

where the factors are

- $r_t$ : excess return of the Berkshire Hathaway stock

- $MKT_t$ (market): excess market return

-  $SMB_t$ (size): small minus big

-  $HML_t$ (value): high book-to-market minus low book-to-market

-  $UMD_t$ (momentum): up minus down

-  $BAB_t$ (betting-against-beta): safe (low beta) minus risky (high beta)

- $QMJ_y$ (quality): quality minus junk

Can we replicate this finding? Fortunately Steve Lihn (on GitHub) already did it.

Data: github.com/slihn/buffetts_alpha_R/archive/master.zip

In [ ]:
# hide
display(Image("images/l1_frazzini_table4heading.PNG"))
display(Image("images/l1_frazzini_table4.PNG"))

- The characteristics of the investment of Warren Buffet: high loadings on replicable factors such as beta, size, value and quality – and a negative loading on momentum.


- At least in this replication of the paper's results (with slightly different data), the intercept is no longer statistically significant – it might still be economically significant!

In [ ]:
from skfin.datasets import load_buffets_data

data = load_buffets_data(cache_dir="data").assign(
    excess_return=lambda x: x["BRK.A"] - x["RF"]
)

In [ ]:
from statsmodels import api

m1 = api.OLS(data["excess_return"], api.add_constant(data["MKT"])).fit()
m1.summary()

In [ ]:
summaries = []
for cols in [
    ["MKT", "SMB", "HML", "UMD"],
    ["MKT", "SMB", "HML", "UMD", "BAB"],
    ["MKT", "SMB", "HML", "UMD", "BAB", "QMJ"],
]:
    m_ = api.OLS(data["excess_return"], api.add_constant(data[cols])).fit()
    summaries += [m_.summary()]

In [ ]:
def prettify_table(tbl):
    df = pd.DataFrame(tbl.tables[1].data)
    idx = df.iloc[1:, 0]
    return pd.DataFrame(
        df.iloc[1:, [1, 3]].astype(float).values,
        index=idx.rename(None),
        columns=["coef", "tstat"],
    ).stack()

In [ ]:
pd.concat([prettify_table(v) for v in summaries], axis=1).fillna(0).round(2)

The coefficients are qualitatively close from the results in the paper -- with the except of the `BAB` coefficients not being statistically significant.

## Industry momentum factor exposure 

In this section, we go back to the Industry momentum backtest and decompose it on the factors as computed by Ken French. 

In [ ]:
returns_data = load_kf_returns(cache_dir="data")
ret = returns_data["Monthly"]["Average_Value_Weighted_Returns"][:"1999"]

In [ ]:
transform_X = lambda x: x.rolling(12).mean().fillna(0)
transform_y = lambda x: x.shift(-1)
features = transform_X(ret)
target = transform_y(ret)

In [ ]:
pnl0 = Backtester(MeanVariance()).train(features, target, ret)
line(pnl0, cumsum=True, title="Industry momentum")

In [ ]:
files = ["F-F_Research_Data_Factors", "F-F_Momentum_Factor"]
df = pd.concat([load_kf_returns(c)["Monthly"] for c in files], axis=1)["1945":"1999"]

In [ ]:
bar(df.corrwith(pnl0), horizontal=True, title="Correlation with industry momentum")

In [ ]:
data = df.join(pnl0.rename("IndustryMom"))

In [ ]:
m = api.OLS(
    data["IndustryMom"], api.add_constant(data.drop("IndustryMom", axis=1))
).fit()

In [ ]:
m.summary()

In [ ]:
line(
    pd.concat({"Stock momentum": df["Mom   "], "Industry momentum": pnl0}, axis=1).pipe(
        lambda x: x.div(x.std())
    ),
    cumsum=True,
)

The main issue with this statistical decomposition is that the estimation is done "full sample". In the next section, we estimate the Momentum loading on rolling windows. 

## Residual pnl 

To run the rolling estimation decomposition, we use the function `fit_predict` used in previous sections. 

In [ ]:
from skfin.backtesting import fit_predict
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
start_date = "1945-01-01"
max_train_size = 60
test_size = 1
params = dict(max_train_size=max_train_size, test_size=test_size, gap=0)
params["n_splits"] = (len(data) - max_train_size) // test_size
cv_ = TimeSeriesSplit(**params)

In [ ]:
pnl_hat, estimator_ = zip(
    *[
        fit_predict(
            estimator=LinearRegression(),
            X=data.drop(["IndustryMom", "RF"], axis=1),
            y=data["IndustryMom"],
            train=train,
            test=test,
            return_estimator=True,
        )
        for train, test in cv_.split(data["IndustryMom"])
    ]
)

In [ ]:
pnl_hat = pd.Series(
    np.concatenate(pnl_hat), index=data["IndustryMom"].index[max_train_size:]
)

In [ ]:
line(
    {
        "pnl0": pnl0[max_train_size:],
        "predict": pnl_hat,
        "residue": pnl0[max_train_size:] - pnl_hat,
    },
    cumsum=True,
    title="Rolling residual decomposition",
)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 5))
line(
    pd.DataFrame([m.intercept_ for m in estimator_], index=pnl_hat.index),
    title="Intercept coefficient",
    ax=ax[0],
    legend=False,
)

line(
    pd.DataFrame(
        [m.coef_ for m in estimator_],
        columns=data.drop(["IndustryMom", "RF"], axis=1).columns,
        index=pnl_hat.index,
    ),
    title="Slope coefficients",
    loc="best",
    ax=ax[1],
)

Over this period, the simple Industry momentum strategy seems to have zero residual relative to other factors. 